In [54]:
import pandas as pd

df = pd.read_csv("diabetes.csv")


In [55]:
# aqui eu vejo as 5 primeiras linhas da tabela para ter uma ideia geral dos dados
df.head() 

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [56]:
# aqui eu verifico informações importantes como tipos de dados e se tem valores faltando
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [57]:
# aqui eu vejo estatísticas básicas das colunas, como média, mínimo, máximo, etc.
df.describe() 

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [58]:

# agora quero identificar se existem valores que são zero (como Insulin ou BloodPressure),
# o que pode indicar dados faltantes

soma_0 = df["Insulin"] == 0

soma_0.sum()

# aqui eu conto quantas vezes o valor da insulina é zero, o que pode indicar ausência de dados
(df["Insulin"] == 0).sum()  

(df["BloodPressure"] == 0).sum()  # aqui eu faço a mesma coisa para a pressão arterial


np.int64(35)

In [59]:
soma = df["BloodPressure"] == 0 

soma.sum()

np.int64(35)

In [60]:
# agora vou criar uma condição para remover essas linhas com zeros inválidos
condicao = (df["BloodPressure"] == 0) | (df["Insulin"] == 0)# aqui eu marco todas as linhas onde insulina ou pressão são zero

condicao.sum()



np.int64(374)

In [61]:
# aqui eu removo as linhas que estavam com valores inválidos (zeros nas colunas importantes)

df_filtrado = df[~condicao]


# operador lógico para negar uma condição em Python é um símbolo que parece um til ~

In [62]:
# aqui eu verifico o novo tamanho da tabela depois da filtragem
df_filtrado.shape

(394, 9)

In [63]:
# agora separo as variáveis que vou usar como entrada (X) e saída (y) no modelo

x = df_filtrado[["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]]  
# aqui eu escolho as colunas que vão ser usadas como entrada para prever o risco de diabetes

y = df_filtrado["Outcome"]  # aqui eu pego a coluna que diz se a pessoa tem diabetes (1) ou não (0), que é o que quero prever



In [64]:
# agora divido os dados em treino e teste

from sklearn.model_selection import train_test_split  # importo a função que divide os dados em treino e teste

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)  
# aqui eu separo 70% dos dados para treino e 30% para teste, de forma aleatória, mas com uma semente fixa (random_state)



In [65]:
# agora vou padronizar os dados para o modelo funcionar melhor
from sklearn.preprocessing import StandardScaler  # importo o escalador padrão para deixar os dados na mesma escala

scaler = StandardScaler()  # crio o objeto que vai fazer a padronização

scaler.fit(x_train)  # aqui eu ajusto o escalador usando os dados de treino (calculo média e desvio padrão)

x_train_scaled = scaler.transform(x_train)  # aplico a padronização nos dados de treino

x_test_scaled = scaler.transform(x_test)  # aplico a mesma padronização nos dados de teste



In [66]:

# agora crio e treino o modelo
from sklearn.linear_model import LogisticRegression  # importo o modelo de regressão logística

model = LogisticRegression()  # crio o objeto do modelo (a "máquina" que vai aprender)

model.fit(x_train_scaled, y_train)  # treino o modelo usando os dados de treino padronizados e os rótulos

# agora gero as previsões com o modelo

y_pred = model.predict(x_test_scaled)  # aqui eu uso o modelo treinado para prever os resultados no conjunto de teste



In [67]:

# agora avalio o desempenho do modelo

from sklearn.metrics import accuracy_score  # importo a função que calcula a acurácia

accuracy = accuracy_score(y_test, y_pred)  # comparo os valores previstos com os reais e calculo quantos acertos o modelo teve

percentage = accuracy * 100  # transformo a acurácia em porcentagem para facilitar a leitura

display(percentage)  # mostro a porcentagem de acerto do modelo


79.83193277310924